In [1]:
#!/usr/bin/env python

from selenium import webdriver
import pandas as pd
import re
from bs4 import BeautifulSoup
from collections import defaultdict
import time
import random

In [3]:
ls

cleaning_prototype.ipynb                geckodriver.log
clean_movie_df.pickle                   movie_info.pickle
Data_Scraping_Prototype.ipynb           movie_stubs.pickle
Data_Scraping_Prototype_Selenium.ipynb  README.md


In [2]:
import pickle 
movie_df = pickle.load(open('clean_movie_df.pickle', 'rb'))

In [99]:
movie_df.link_stub

10 Cloverfield Lane            /release/rl1329956353/?ref_=bo_yld_table_44
10,000 BC                       /release/rl977372673/?ref_=bo_yld_table_26
102 Dalmatians                 /release/rl927041025/?ref_=bo_yld_table_137
102 Dalmatians                  /release/rl927041025/?ref_=bo_yld_table_44
12 Rounds                      /release/rl440501761/?ref_=bo_yld_table_147
                                                  ...                     
xXx                            /release/rl1450870273/?ref_=bo_yld_table_15
xXx: Return of Xander Cage     /release/rl1904182785/?ref_=bo_yld_table_63
xXx: State of the Union        /release/rl1467647489/?ref_=bo_yld_table_96
Æon Flux                      /release/rl1212188161/?ref_=bo_yld_table_270
Æon Flux                      /release/rl1212188161/?ref_=bo_yld_table_102
Name: link_stub, Length: 4195, dtype: object

In [24]:
def get_cast_info(link):
    
    base_url = "https://www.boxofficemojo.com"
    url = base_url+link
    #linux browser = webdriver.Firefox()
    browser = webdriver.Chrome(executable_path='/Applications/chromedriver')
    browser.get(url)
    time.sleep(0.5*random.random())
    browser.find_element_by_link_text('Title Summary').click()
    browser.find_element_by_link_text('Cast and Crew').click()

    soup = BeautifulSoup(browser.page_source)
               
    crew_dict = defaultdict(list)
    
    #find Table Element 
    tables = soup.find_all('table')

    crew_rows = tables[0].find_all('tr')
    for row in crew_rows[1:]:
        curr_row = row.find_all('td')
        role = curr_row[1].text.rstrip()
        name = curr_row[0].text.rstrip()
        #add to crew dictionairy
        crew_dict[role].append(name)


    actor_rows = tables[1].find_all('tr')
    for row in actor_rows[1:]:
        curr_row = row.find_all('td')
        name = curr_row[0].text.rstrip()
        crew_dict['Actors'].append(name)

    cast_dict = {}
    title = soup.find('h1').text
    cast_dict['title'] = title
    
    for key in crew_dict:
        cast_dict[key] = ', '.join(crew_dict[key])
        
    browser.quit()
    return cast_dict
    

In [29]:
cast_info_list = []

In [30]:
c = 0
for link in movie_df.link_stub[:1500]:
    try:
        cast_info_list.append(get_cast_info(link))
        browser.quit()
    except:
        c+=1
        print('{} rows missed', end="\r", flush=True)
    print('{} rows completed'.format(len(cast_info_list)), end='\r', flush=True)

In [31]:
import pickle
with open('cast_info_list_start_to_1500.pickle', 'wb') as to_write:
    pickle.dump(cast_info_list, to_write)

In [32]:
len(cast_info_list)

1499

In [33]:
ls

Data_Scraping_Prototype.ipynb           cleaning_prototype.ipynb
Data_Scraping_Prototype_Selenium.ipynb  geckodriver.log
Feature Engineering.ipynb               movie_info.pickle
README.md                               movie_stubs.pickle
cast_info_list1500_to_end.pickle        processed_movie_df.pickle
cast_info_list_start_to_1500.pickle     usholidays.csv
clean_movie_df.pickle


In [34]:
end = pickle.load(open('cast_info_list1500_to_end.pickle', 'rb'))

In [35]:
len(end)

992

In [48]:
len(movie_df)

2492

In [38]:
type(end)

list

In [39]:
cast_info_list.append(end)

In [40]:
len(cast_info_list)

1500

In [49]:
cast_info = cast_info_list + end

In [ ]:
with open('cast_info.pickle', '')